In [8]:
import os
import xml.etree.ElementTree as ET
from openpyxl import Workbook

# Caminho da pasta onde os XMLs estão armazenados
pasta_xml = 'NFs'

# Namespace da NF-e 
ns = {'nfe': 'http://www.portalfiscal.inf.br/nfe'}

# Cria um novo arquivo Excel
wb = Workbook()
ws = wb.active
ws.title = "Itens Notas Fiscais"

# Adiciona o cabeçalho na planilha
ws.append(["utilizacao", "emitente", "destinatario", "cnpj", "PO", "prod", "quantidade", "preço", "local de entrega", "NF",])

# Itera sobre todos os arquivos na pasta
for arquivo in os.listdir(pasta_xml):
    if arquivo.endswith('.xml'):
        caminho_xml = os.path.join(pasta_xml, arquivo)

        # Carrega o arquivo XML e transformar em uma arvore de elementos manipulaveis
        tree = ET.parse(caminho_xml)
        root = tree.getroot()


        # Acessa os itens da nota fiscal
        destinatario = root.find('.//nfe:dest/nfe:xNome', ns).text

        emitente = root.find('.//nfe:emit/nfe:xNome', ns).text

        cnpj = root.find('.//nfe:dest/nfe:CNPJ', ns).text

        texto_informacoes_XML = root.find('.//nfe:infAdic/nfe:infCpl', ns).text
        indice_PO_incio = texto_informacoes_XML.find('Ped. Cliente:') + 13
        indice_PO_final = indice_PO_incio + 5
        pedido_de_compra = texto_informacoes_XML[indice_PO_incio:indice_PO_final]
        
        indice_local_inicio = texto_informacoes_XML.find('-Nome do local de entrega-')
        indice_local_final = indice_local_inicio + 7
        local_de_entrega = texto_informacoes_XML[indice_local_inicio:indice_local_final]
        
        nf = root.find('.//nfe:ide/nfe:nNF', ns).text

        dados_gerais = root.find('.//nfe:ide', ns)
        utilizacao = dados_gerais.find('.//nfe:natOp', ns).text

        itens = root.findall('.//nfe:det', ns)

        # Verifica se há itens no XML
        if not itens:
            print(f'Nenhum item encontrado no arquivo: {arquivo}')
            continue
        
        # Itera sobre os itens e salva as informações na planilha
        for item in itens:
            nome_produto = item.find('.//nfe:xProd', ns).text
            quantidade = item.find('./nfe:prod/nfe:qCom', ns).text
            preco = item.find('.//nfe:prod/nfe:vUnCom', ns).text

            # Adiciona a linha na planilha
            ws.append([utilizacao, emitente, destinatario, cnpj, pedido_de_compra, nome_produto, quantidade, preco, local_de_entrega, nf])

# Salva o arquivo Excel
wb.save("itens_notas_fiscais.xlsx")
print("Dados salvos no arquivo 'itens_notas_fiscais.xlsx'")


Dados salvos no arquivo 'itens_notas_fiscais.xlsx'
